# Credit Risk — Fannie Mae (Q1 2022) Preprocessing

Goal: Build a compact, model-ready table with one row per loan at origination and a binary label = default within 12 months.

### Why this structure?

- I model a decision made at loan approval time, so we only keep features known at origination (e.g., credit score, DTI, LTV, purpose, property type, state, etc.).

- The target (label) is computed from post-origination performance but only to define the outcome, never as input features.

### What this notebook does:

1. Load the large pipe-delimited file (no header).

2. Rename c0…c109 → official field names for the columns we need.

3. Parse dates (MMYYYY).

4. Compute months since origination for each reporting row.

5. Create the label default_12m = 1 if within months 0–11 the loan is 90+ DPD or hits a distressed zero-balance code (e.g., short sale / deed-in-lieu / REO).

6. Select origination features and collapse to one row per loan.

7. Export a tidy CSV for modeling.

In [1]:
# --- Imports + read sample (adjust nrows or remove to read full file on Kaggle) ---
import os
import pandas as pd
from datetime import datetime

data_dir = "/kaggle/input/q1-2022-fanniemae"
file_path = os.path.join(data_dir, "2022Q1.csv")  # adjust if needed

df = pd.read_csv(
    file_path,
    sep="|",
    header=None,
    dtype=str,           # keep as string to avoid mixed-type issues
    on_bad_lines="skip",
    low_memory=False,
    nrows=2_000_000
)

df.columns = [f"c{i}" for i in range(df.shape[1])]
df.shape, df.head(3)


((2000000, 110),
     c0            c1      c2 c3     c4                            c5   c6  \
 0  NaN  000130357804  012022  R  Other  Lakeview Loan Servicing, LLC  NaN   
 1  NaN  000130357804  022022  R  Other  Lakeview Loan Servicing, LLC  NaN   
 2  NaN  000130357804  032022  R  Other  Lakeview Loan Servicing, LLC  NaN   
 
       c7     c8         c9  ... c100 c101 c102 c103 c104 c105 c106 c107 c108  \
 0  3.375  3.375  255000.00  ...  NaN    7    N  NaN  NaN    7  NaN  NaN    7   
 1  3.375  3.375  255000.00  ...  NaN    7    N  NaN  NaN    7  NaN  NaN    7   
 2  3.375  3.375  255000.00  ...  NaN    7    N  NaN  NaN    7  NaN  NaN    7   
 
   c109  
 0  NaN  
 1  NaN  
 2  NaN  
 
 [3 rows x 110 columns])

### Minimal official Mapping 
This maps only the columns I actually need for organisation features + label.

In [2]:
# --- Minimal mapping for features + label creation ---
col_map = {
    "c0":  "REFERENCE_POOL_ID",
    "c1":  "LOAN_ID",
    "c2":  "MONTHLY_REPORTING_PERIOD",  # MMYYYY
    "c3":  "CHANNEL",                   # R/C/B/etc.
    "c4":  "SELLER_NAME",
    "c5":  "SERVICER_NAME",
    "c7":  "ORIG_INTEREST_RATE",
    "c8":  "CURR_INTEREST_RATE",
    "c9":  "ORIG_UPB",
    "c11": "CURRENT_ACTUAL_UPB",
    "c12": "ORIG_LOAN_TERM",
    "c13": "ORIGINATION_DATE",          # MMYYYY
    "c14": "FIRST_PAYMENT_DATE",        # MMYYYY
    "c18": "MATURITY_DATE",             # MMYYYY
    "c19": "ORIG_LTV",
    "c20": "ORIG_CLTV",
    "c21": "NUM_BORROWERS",
    "c22": "DTI",
    "c23": "BORR_CREDIT_SCORE",
    "c24": "CO_BORR_CREDIT_SCORE",
    "c25": "FIRST_TIME_HB_FLAG",
    "c26": "LOAN_PURPOSE",
    "c27": "PROPERTY_TYPE",
    "c28": "NUM_UNITS",
    "c29": "OCCUPANCY_STATUS",
    "c30": "PROPERTY_STATE",
    "c33": "MI_PCT",
    "c35": "AMORTIZATION_TYPE",         # FRM/ARM
    "c36": "PREPAYMENT_PENALTY_FLAG",
    "c37": "INTEREST_ONLY_FLAG",
    "c40": "CURR_DELINQ_STATUS",        # '00','01','02','03',...,'99','XX'
    "c41": "PAYMENT_HISTORY_24MO",
    "c44": "ZERO_BALANCE_CODE",         # '01','02','03','06','09','96',...
    "c45": "ZERO_BALANCE_EFF_DATE",     # MMYYYY
    "c51": "LAST_PAID_INSTALLMENT_DATE",
    "c52": "FORECLOSURE_DATE",
    "c53": "DISPOSITION_DATE",
}

df = df.rename(columns=col_map)
df[list(col_map.values())].head(3)


,REFERENCE_POOL_ID,LOAN_ID,MONTHLY_REPORTING_PERIOD,CHANNEL,SELLER_NAME,SERVICER_NAME,ORIG_INTEREST_RATE,CURR_INTEREST_RATE,ORIG_UPB,CURRENT_ACTUAL_UPB,...,AMORTIZATION_TYPE,PREPAYMENT_PENALTY_FLAG,INTEREST_ONLY_FLAG,CURR_DELINQ_STATUS,PAYMENT_HISTORY_24MO,ZERO_BALANCE_CODE,ZERO_BALANCE_EFF_DATE,LAST_PAID_INSTALLMENT_DATE,FORECLOSURE_DATE,DISPOSITION_DATE
0,NaN,000130357804,012022,R,Other,"Lakeview Loan Servicing, LLC",3.375,3.375,255000.00,255000.00,...,N,N,NaN,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX00,N,NaN,NaN,NaN,NaN,NaN
1,NaN,000130357804,022022,R,Other,"Lakeview Loan Servicing, LLC",3.375,3.375,255000.00,255000.00,...,N,N,NaN,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX0000,N,NaN,NaN,NaN,NaN,NaN
2,NaN,000130357804,032022,R,Other,"Lakeview Loan Servicing, LLC",3.375,3.375,255000.00,255000.00,...,N,N,NaN,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX000000,N,NaN,NaN,NaN,NaN,NaN


### Date parsing + month math 

In [3]:
# --- Parse MMYYYY into pandas period for easy month arithmetic ---
def parse_mmyyyy_to_period(s):
    try:
        # Ensure length is 6 like '012022'; guard NaNs
        s = str(s)
        if len(s) == 6 and s.isdigit():
            return pd.Period(s, freq='M')
    except Exception:
        pass
    return pd.NaT

date_cols = [
    "MONTHLY_REPORTING_PERIOD",
    "ORIGINATION_DATE",
    "FIRST_PAYMENT_DATE",
    "MATURITY_DATE",
    "ZERO_BALANCE_EFF_DATE",
    "FORECLOSURE_DATE",
    "DISPOSITION_DATE",
    "LAST_PAID_INSTALLMENT_DATE",
]

for c in date_cols:
    if c in df.columns:
        df[c] = df[c].apply(parse_mmyyyy_to_period)

# months since orig = (reporting_period - origination_date) in months
def months_between(p1, p0):
    if pd.isna(p1) or pd.isna(p0):
        return pd.NA
    return (p1.year - p0.year) * 12 + (p1.month - p0.month)

df["months_since_orig"] = df.apply(
    lambda r: months_between(r.get("MONTHLY_REPORTING_PERIOD"), r.get("ORIGINATION_DATE")),
    axis=1
)
df["in_first_12m"] = df["months_since_orig"].apply(lambda x: (x is not pd.NA) and (0 <= x <= 11))
df["at_origination_row"] = df["months_since_orig"].apply(lambda x: x == 0)


### Label: default within 12months
- If within months 0–11 the loan ever has 90+ DPD (CURR_DELINQ_STATUS >= '03') → default=1

- Or within months 0–11, ZERO_BALANCE_CODE in {'02','03','09'} → (third-party sale, short sale, deed-in-lieu/REO) → default=1

- Else default=0

In [4]:
# --- Helpers to evaluate delinquency / ZB codes safely on strings ---
def is_90dpd_or_more(status):
    """
    status: string like '00','01','02','03','09','XX','R','  ' etc.
    Treat invalid or special codes ('XX','  ','R') as not 90+ by default.
    """
    if status is None:
        return False
    s = str(status).strip()
    # quick accept for numeric 2+ digits like '10','11'
    if s.isdigit():
        try:
            return int(s) >= 3
        except Exception:
            return False
    return False

DISTRESSED_ZB = {"02", "03", "09"}  # 02: Third Party Sale, 03: Short Sale, 09: Deed-in-Lieu/REO

def is_distressed_zb(zb):
    if zb is None:
        return False
    s = str(zb).strip()
    return s in DISTRESSED_ZB

# --- Compute label per loan over the first 12 months window ---
mask_12m = df["in_first_12m"] == True
win = df.loc[mask_12m, ["LOAN_ID", "CURR_DELINQ_STATUS", "ZERO_BALANCE_CODE"]].copy()
win["flag_90dpd"] = win["CURR_DELINQ_STATUS"].apply(is_90dpd_or_more)
win["flag_distressed_zb"] = win["ZERO_BALANCE_CODE"].apply(is_distressed_zb)
agg = win.groupby("LOAN_ID", as_index=False)[["flag_90dpd", "flag_distressed_zb"]].any()
agg["default_12m"] = (agg["flag_90dpd"] | agg["flag_distressed_zb"]).astype(int)
agg = agg[["LOAN_ID", "default_12m"]]

agg.head(5), agg["default_12m"].mean()  # quick sanity: prevalence


(        LOAN_ID  default_12m
 0  000130357804            0
 1  000130357805            0
 2  000130357806            0
 3  000130357807            0
 4  000130357808            0,
 0.0)

### One row per loan (origination features)

I take the origination row (months_since_orig == 0) as the canonical source of origination-time features.

In [5]:
# --- Pick origination snapshot rows ---
orig_cols = [
    "LOAN_ID",
    "ORIGINATION_DATE",
    "FIRST_PAYMENT_DATE",
    "ORIG_INTEREST_RATE",
    "ORIG_UPB",
    "ORIG_LOAN_TERM",
    "ORIG_LTV",
    "ORIG_CLTV",
    "DTI",
    "BORR_CREDIT_SCORE",
    "CO_BORR_CREDIT_SCORE",
    "NUM_BORROWERS",
    "FIRST_TIME_HB_FLAG",
    "LOAN_PURPOSE",
    "PROPERTY_TYPE",
    "NUM_UNITS",
    "OCCUPANCY_STATUS",
    "PROPERTY_STATE",
    "MI_PCT",
    "AMORTIZATION_TYPE",
    "CHANNEL",
    # you may drop SERVICER/SELLER if you prefer
    # "SELLER_NAME",
    # "SERVICER_NAME",
]

orig = df.loc[df["at_origination_row"] == True, orig_cols].drop_duplicates(subset=["LOAN_ID"])
orig.shape, orig.head(3)


((11196, 21),
           LOAN_ID ORIGINATION_DATE FIRST_PAYMENT_DATE ORIG_INTEREST_RATE  \
 0    000130357804          2022-01            2022-03              3.375   
 273  000130357811          2022-01            2022-03              3.250   
 351  000130357813          2022-01            2022-03              3.000   
 
       ORIG_UPB ORIG_LOAN_TERM ORIG_LTV ORIG_CLTV DTI BORR_CREDIT_SCORE  ...  \
 0    255000.00            360       72        72  43               703  ...   
 273  184000.00            360       80        80  35               792  ...   
 351  447000.00            360       53        53  43               691  ...   
 
     NUM_BORROWERS FIRST_TIME_HB_FLAG LOAN_PURPOSE PROPERTY_TYPE NUM_UNITS  \
 0               1                  N            P            SF         1   
 273             1                  N            R            SF         1   
 351             1                  N            C            PU         1   
 
     OCCUPANCY_STATUS PROPERTY_STATE MI_

### Merge features + label → final training table

In [6]:
# --- Merge label onto features (left keep: only loans with an origination row) ---
final = orig.merge(agg, on="LOAN_ID", how="left")
final["default_12m"] = final["default_12m"].fillna(0).astype(int)  # if no window rows, treat as 0

# Optional: cast numeric-ish fields to numeric (coerce)
num_cols = [
    "ORIG_INTEREST_RATE", "ORIG_UPB", "ORIG_LOAN_TERM", "ORIG_LTV", "ORIG_CLTV",
    "DTI", "BORR_CREDIT_SCORE", "CO_BORR_CREDIT_SCORE", "NUM_BORROWERS", "NUM_UNITS", "MI_PCT"
]
for c in num_cols:
    if c in final.columns:
        final[c] = pd.to_numeric(final[c], errors="coerce")

final.shape, final.head(5)


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


((11196, 22),
         LOAN_ID ORIGINATION_DATE FIRST_PAYMENT_DATE  ORIG_INTEREST_RATE  \
 0  000130357804          2022-01            2022-03               3.375   
 1  000130357811          2022-01            2022-03               3.250   
 2  000130357813          2022-01            2022-03               3.000   
 3  000130357817          2022-01            2022-03               3.125   
 4  000130357819          2022-01            2022-03               3.250   
 
    ORIG_UPB  ORIG_LOAN_TERM  ORIG_LTV  ORIG_CLTV  DTI  BORR_CREDIT_SCORE  ...  \
 0  255000.0             360        72         72   43              703.0  ...   
 1  184000.0             360        80         80   35              792.0  ...   
 2  447000.0             360        53         53   43              691.0  ...   
 3  147000.0             360        95         95   19              768.0  ...   
 4  400000.0             360        42         42   28              776.0  ...   
 
    FIRST_TIME_HB_FLAG  LOAN_PURPO

### Save CSV

In [7]:
out_path = "/kaggle/working/fannie_q1_2022_labeled.csv"
final.to_csv(out_path, index=False)
out_path, final["default_12m"].value_counts(dropna=False)


('/kaggle/working/fannie_q1_2022_labeled.csv',
 default_12m
 0    11196
 Name: count, dtype: int64)